# Load Pickle Data

In [1]:
import pandas as pd
from pathlib import Path

data_dir = Path('/mnt/data/factcheck/tweets/similarity_web_data')

print(pd.__version__)
# df = pd.read_pickle(data_dir / 'df_titles.pkl')

1.4.2


In [2]:
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from tqdm.auto import tqdm
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [3]:
import json

def load_dict(flnm):
    with open(flnm, 'rt') as f:
        data = json.load(f)
    data = {int(k): v for k, v in data.items()}
    return data

all_to_text = load_dict(data_dir / 'all_text.json')
all_to_vecnos = load_dict(data_dir / 'all_vecno.json')
claims = list(all_to_text.values())

print(len(claims))

203057


In [4]:
claims = list(all_to_text.values())
print('length of claims (texts):', len(claims))

v_torch = torch.load(data_dir / 'all_tensors_pca.pt')

v_numpy = v_torch.numpy()
print('corresponding shape of embeddings:', v_numpy.shape)

length of claims (texts): 203057
corresponding shape of embeddings: (203057, 512)


In [5]:
from sklearn.decomposition import PCA

n = np.linalg.norm(v_numpy, axis=1, keepdims=True)
vectors = v_numpy / n
# vectors = v_numpy

check = (vectors**2).sum(axis=1)

print(check.mean(), check.std())
print(vectors.shape)

163.97655 32.880253
(203057, 512)


# Choose vectors

In [6]:
rng = np.random.default_rng()

# sub = rng.choice(len(vectors), 50000, replace=False)
# sub.sort()

sub = np.arange(len(vectors))

subvectors = vectors[sub]
subclaims = [claims[i] for i in sub]

# Pre-Compute Clustering

## Pre-computed DBSCAN clustering

In [ ]:
import time

from sklearn.cluster import DBSCAN

out_path = Path('/mnt/data/factcheck/tweets/clustering/indices')
    
def compute_dbscan(vectors, name, epsilons, min_samples=5):
    for eps in epsilons:
        print(f'\r{eps}', end='')
        db = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine', n_jobs=-1)
        clustering = db.fit(vectors)
        with open(out_path / f'{name}_{int(eps*100)}.npy', 'wb') as f:
            np.save(f, clustering.labels_)

# compute_dbscan(subvectors, 'eps', [0.4, 0.45, 0.5, 0.55, 0.6]) 

## Compute 10 batches of 1000, 1500, ... 10000 clusters

In [7]:
import time
from sklearn.cluster import KMeans, MiniBatchKMeans
from pathlib import Path
out_path = Path('/mnt/data/factcheck/tweets/clustering/indices')

def compute_kmeans(vectors, n_clusters, num):
    start = time.time()
    km = MiniBatchKMeans(n_clusters=n_clusters, batch_size=2048)
    clustering = km.fit(vectors)
    print(n_clusters, 'computed in', time.time() - start)
    cc = np.array(clustering.labels_, dtype=np.int32)
    with open(out_path / f'kmeans_bunn{num}_{n_clusters}.npy', 'wb') as f:
        np.save(f, cc, allow_pickle=False)

# print(len(subvectors))
# for num in range(10):
#     for n_clusters in range(1000, 10500, 500):
#         compute_kmeans(subvectors, n_clusters, num)

for n_clusters in range(16000, 21000, 1000):
    compute_kmeans(subvectors, n_clusters, 1)

16000 computed in 2018.1455085277557
17000 computed in 2270.599839448929
18000 computed in 2532.303883075714
19000 computed in 2820.4874000549316
20000 computed in 3091.310382604599


In [ ]:
# amd 4
1000 computed in 20.495242834091187
2000 computed in 41.9652955532074
3000 computed in 70.37838864326477
4000 computed in 91.49673748016357
5000 computed in 115.1496798992157
6000 computed in 133.79012608528137
7000 computed in 414.39016914367676
8000 computed in 531.0927817821503
9000 computed in 667.9861640930176
10000 computed in 826.4574625492096
11000 computed in 970.5446286201477
12000 computed in 1175.2351398468018
13000 computed in 1376.2166557312012
14000 computed in 1582.2085995674133
15000 computed in 1792.3280866146088

In [11]:
compute_kmeans(subvectors, 20000, 0)

20000 computed in 3085.6187541484833


bunn 8 proc
1000 computed in 32.97154211997986
2000 computed in 60.52177429199219
3000 computed in 113.83817052841187
4000 computed in 110.82460618019104
5000 computed in 197.31179928779602
6000 computed in 235.99499440193176
7000 computed in 768.0193598270416
8000 computed in 973.5231347084045
9000 computed in 1168.9018733501434
10000 computed in 1487.1080164909363
11000 computed in 1600.0052433013916
12000 computed in 1871.2723133563995
13000 computed in 2155.8638939857483
14000 computed in 2493.8044843673706
15000 computed in 2866.7826178073883
16000 computed in 3246.5067250728607
17000 computed in 3718.830470085144
18000 computed in 4234.489676237106
19000 computed in 5287.410274982452

8 processors

1000 computed in 25.91518521308899
2000 computed in 52.79446220397949
3000 computed in 75.97883629798889
4000 computed in 110.1669819355011
5000 computed in 127.25340867042542
6000 computed in 239.02578377723694
7000 computed in 945.675986289978
8000 computed in 983.9306406974792
9000 computed in 1043.3538360595703
10000 computed in 1280.3090834617615
11000 computed in 1524.3385672569275
12000 computed in 1815.0138640403748
13000 computed in 2089.463007926941
14000 computed in 2337.3573043346405
15000 computed in 2574.91379404068
16000 computed in 4522.249264001846
17000 computed in 3531.9385907649994
18000 computed in 4135.081263303757
19000 computed in 4601.594769716263
20000 computed in 5124.634221076965

# Play with K-Means data

In [33]:
out_path = Path('/mnt/data/factcheck/tweets/clustering/indices')
c = clustering.labels_
print(c.shape)
cc = np.array(c, dtype=np.int32)
np.save(out_path / 'save.npy', cc, allow_pickle=False)

(203057,)


In [32]:
l = np.load(out_path / 'save.npy', allow_pickle=False)
l.shape

(203057,)

# Use Lemmatized Data. Lemma Function.

In [ ]:
import json
from pathlib import Path
import pandas as pd
from itertools import chain
from collections import Counter

lemma_path = Path('/mnt/data/factcheck/tweets/clustering/lemmatized_all.json')
with open(lemma_path, 'rt') as f:
    lemmas = json.load(f)

sublemmas = [lemmas[i] for i in sub]

pd = pd.read_csv('/mnt/data/factcheck/tweets/clustering/syn2015_word_utf8.csv',
                 sep=';', header=None)
source = pd[1].tolist()
most_frequent = source[:300] + ['the', 'and']

def get_main_lemmas(indices):
    used_lemmas = [sublemmas[i] for i in indices]
    flattened = list(chain.from_iterable(used_lemmas))
    filtered = [w for w in flattened if len(w) > 2]
    counter = Counter(filtered)
    most_common = counter.most_common(10)
    fmc = [item[0] for item in most_common 
           if item[1]>=len(indices)//4 and item[0] not in most_frequent]
    return fmc

# Load Clustering and Use It

In [ ]:
! cd /mnt/data/factcheck/tweets/clustering/indices; ls

In [8]:
from pathlib import Path

path = Path('/mnt/data/factcheck/tweets/clustering/indices')

def load_clusterings(name, epses):
    clusterings = {}
    for eps in epses:
        with open(path / f'{name}_{int(eps*100)}.npy', 'rb') as f:
            arr = np.load(f)
            print((arr >= 0).sum(), arr.max())
            clusterings[eps] = arr
    return clusterings

clusterings = load_clusterings('eps', [0.4, 0.45, 0.5, 0.55, 0.6])

66070 331
113865 161
165245 28
195289 0
202584 0


In [9]:
def get_groups(label_arr):
    res = [(label_arr==i).nonzero()[0] for i in range(label_arr.max())]
    return sorted(res, key=lambda x: -len(x))

def get_texts(indices):
    return [subclaims[i] for i in indices]

groups = get_groups(clusterings[0.45])
lengths = [len(item) for item in groups]
print(lengths)

# texts = get_texts(g[2][:20])
# print(texts)

[112972, 29, 23, 21, 16, 13, 12, 12, 12, 11, 11, 11, 11, 10, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1]


# Use Lemmatized

In [ ]:
for i, group in enumerate(groups):
    lemmas = get_main_lemmas(group)
    print(f'{i:02d}', f'{len(group):02d}', *lemmas)
    if len(group) < 10:
        break
        
def print_tweets(i):
    texts = get_texts(groups[i])
    for text in texts:
        print(text)

In [ ]:
print_tweets(14)

# Load Everything for Matrix Multiplication

In [ ]:
from pathlib import Path

annotations_path = Path('/mnt/data/factcheck/tweets/similarity_web_annotations')

annotations = [json.loads(p.read_text()) 
               for p in annotations_path.iterdir() 
               if p.is_file() and p.name.endswith('json')]

In [ ]:
twitter_ids = [ [int(tid) for tid in group['included']] for group in annotations ]
vecnos = [ [all_to_vecnos[tid] for tid in group] for group in twitter_ids ]
total_vecnos = len(all_to_text)
torch_vecnos = [torch.tensor(vn).to(device) for vn in vecnos]

In [ ]:
labels = np.zeros(total_vecnos)
labels[:] = -1
for i, vn in enumerate(vecnos):
    labels[vn] = i
    
labels = torch.tensor(labels, dtype=int).reshape(-1,1)
labels.shape, labels.dtype

In [ ]:
class_tensors = []
for i in range(labels.max()+1):
    c = (labels==i).view(-1).nonzero()
    class_tensors.append(c)

# Train Matrix Multiplication

In [ ]:
import torch

from torch.utils.data import TensorDataset, Dataset, DataLoader

In [ ]:
vecs = torch.load(data_dir / 'all_tensors_pca.pt')
vecs /= vecs.norm(dim=1)[:, None]
print(np.linalg.norm(vecs, axis=1).mean(), np.linalg.norm(vecs, axis=1).std())
print(vecs.shape)
dataset = TensorDataset(vecs.to(device), labels.to(device))

In [ ]:
from dataclasses import dataclass
import numpy as np
import pytorch_lightning as pl

@dataclass
class FullyRandomSampler(torch.utils.data.sampler.Sampler):
    total_items: int
    bags: int
    bag_size: int
    batches: int = 1
    seed: int = 42

    def __iter__(self):
        for _ in range(self.batches):
            yield torch.randint(low=0, high=self.total_items, size=(self.bags, self.bag_size))

    def __len__(self):
        return self.batches

In [ ]:
@dataclass
class ClassRandomSampler(torch.utils.data.sampler.Sampler):
    vec_nos: list[torch.Tensor]
    bags: int
    bag_size: int
    batches: int = 1
    seed: int = 42

    def __iter__(self):
        for _ in range(self.batches):
            which = torch.randint(low=0, high=len(self.vec_nos), size=(self.bags,))
            bags = []
            for w in which:
                w = w.item()
                indices = torch.randint(low=0, high=len(self.vec_nos[w]), size=(self.bag_size,), device=device)
                bags.append(self.vec_nos[w][indices].squeeze(dim=-1))
            yield torch.stack(bags, dim=0)
        
    def __len__(self):
        return self.batches

In [ ]:
random_bags, class_bags = 20, 100

frs = FullyRandomSampler(total_items=len(torch_vecnos), bags=random_bags, bag_size=100)
crs = ClassRandomSampler(vec_nos=class_tensors, bags=class_bags, bag_size=100)

fdl = DataLoader(dataset, batch_sampler=frs)
cdl = DataLoader(dataset, batch_sampler=crs)

In [ ]:
@dataclass
class LTArgs:
    lr: float = 0.0001

    
class LinearTransform(pl.LightningModule):
    def __init__(self, args):
        super().__init__()
        
        self.args = args
        
        m = torch.eye(512)
        r = torch.rand(512,512) * 0.01
        
        self.W = torch.nn.Parameter(m + r)

        self.save_hyperparameters("args")

    def train_dataloader(self):
        return {"full": fdl, "class": cdl}

    def forward_loss(self, fullbags, classbags):
        # no_of_bags, bag_size, vector_dim
        
        fullbags = self.W[None, None, :, :] @ fullbags[:, :, :, None]
        classbags = self.W[None, None, :, :] @ classbags[:, :, :, None]
        
        fullbags = fullbags.squeeze(dim=-1)
        classbags = classbags.squeeze(dim=-1)
        
        # normalize, so that scalar product == cosine similarity
        fullbags_norm = fullbags / fullbags.norm(dim=-1)[..., None]
        classbags_norm = classbags / classbags.norm(dim=-1)[..., None]
        
#         fcenters = fullbags_norm.mean(dim=1, keepdims=True)
#         ccenters = classbags_norm.mean(dim=1, keepdims=True)
        
#         fdists = (fullbags_norm*fcenters).abs().mean(dim=1).sum()
#         cdists = (classbags_norm*ccenters).abs().mean(dim=1).sum()

        # distance of every item with every other item within the bag    
        fdists = (fullbags_norm[:, None, :, :] * 
                  fullbags_norm[:, :, None, :]).abs().sum(dim=3).mean(dim=(1,2)).sum(dim=0)
    
        cdists = (classbags_norm[:, None, :, :] * 
                  classbags_norm[:, :, None, :]).abs().sum(dim=3).mean(dim=(1,2)).sum(dim=0)
        
        # we want class vectors as close and random vectors as far as possible
        return cdists - fdists

    def training_step(self, batch, batch_idx):
        fullbags, classbags = batch['full'][0], batch['class'][0]
        loss = self.forward_loss(fullbags, classbags)
        self.log("loss", loss, prog_bar=True,
                 logger=True, on_step=True, on_epoch=True)
        return loss
    
    def predict_step(self, batch, batch_idx):
        return self(batch[0])

    def forward(self, vectors):
        assert len(vectors.shape) == 2
        assert vectors.shape[1] == 512
        mult = self.W[None, :, :] @ vectors[:, :, None]
        mult = mult.squeeze(dim=2)
        return mult / mult.norm(dim=-1)[..., None]

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.args.lr)
        return optimizer

In [ ]:
ltargs = LTArgs()
linear_transform = LinearTransform(ltargs)

root_dir = Path("/mnt/data/factcheck/tweets/clustering/")

trainer = pl.Trainer(gpus=1, log_every_n_steps=1, max_epochs=5000,
                     default_root_dir=root_dir)
trainer.fit(linear_transform)
trainer.save_checkpoint(root_dir / 'saved_models' / 
                        f'linear_transform_{random_bags}_{class_bags}')

In [ ]:
vec_dataset = TensorDataset(vecs.to(device))
vec_loader = DataLoader(vec_dataset, batch_size=128000)
batch_predictions = trainer.predict(linear_transform, vec_loader)
predictions = torch.cat(batch_predictions, dim=0).cpu().numpy()
predictions.shape

# Work with Transformed Data

In [ ]:
epsilons = [0.1, 0.2, 0.3, 0.4]
compute_dbscan(predictions, f'transf_{random_bags}_{class_bags}', epsilons, min_samples=10) 

In [ ]:
clusterings = load_clusterings(f'transf_{random_bags}_{class_bags}', epsilons)

In [ ]:
groups = get_groups(clusterings[0.3])
lengths = [len(item) for item in groups]
print(lengths)

In [ ]:
for i, group in enumerate(groups):
    lemmas = get_main_lemmas(group)
    print(f'{i:02d}', f'{len(group):02d}', *lemmas)
    if len(group) < 5:
        break
        
def print_tweets(i):
    texts = get_texts(groups[i])
    for text in texts:
        print(text)

In [ ]:
print_tweets(26)